# The following code is responsible for converting MIDI files into text notation.

The whole code is based on the following model: https://keunwoochoi.wordpress.com/2016/02/23/lstmetallica/ 

# I found a way to use the original python library!

The original code was developed using Python2 and the Python-midi library, which is not available for Python3.

Basically i found a Python3 compatible version of the original library.


Reference here: https://github.com/jameswenzel/mydy/blob/master/src/Containers.py
https://github.com/jameswenzel/Fractal-Midi/blob/master/script.py
https://github.com/vishnubob/python-midi

## Opening MIDI file
Basically the next cell opens and reads a dummy MIDI file written by me.

I'm also setting the  ```resolution ``` parameter to 480. This parameter is equivalent to **PPQ** in MIDI files.

**PPQ** (*Pulse per Quarter Note*) is a fixed value which sets the number of pulses contained in a quarter note, it's like a "sampling" frequency.

Each quarter note (no matter what is the original speed of the song) will contain 480 pulses. Then these pulses are converted into actual playback using the **Tempo** information of the MIDI file (obviously a quarter note at 100 BPM is slower than a quarter note at 130 BPM)

In [32]:
from mydy import Events, FileIO, Containers, Constants
test=FileIO.read_midifile('loop 1.mid') #returns a Pattern with the MIDI file information (resolution ecc...), based on documentation https://github.com/jameswenzel/mydy/blob/master/src/FileIO.py

test.resolution=480 #qui sto settando quanti tick ho in una quarter note. Quindi ogni quarter note avra 480 ticks.
print(test) #seems that changing the BPM doesn't influence the ticks.
#Resolution is the same as PPQ


mydy.Pattern(format=0, resolution=480, tracks=\
[mydy.Track(relative: True\
  [mydy.TrackNameEvent(tick=0.0, text='', data=[]),
   mydy.TimeSignatureEvent(tick=0.0, data=[3, 2, 36, 8]),
   mydy.TimeSignatureEvent(tick=0.0, data=[3, 2, 36, 8]),
   mydy.NoteOnEvent(tick=1920.0, channel=0, data=[36, 100]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[42, 100]),
   mydy.NoteOffEvent(tick=120.0, channel=0, data=[36, 64]),
   mydy.NoteOffEvent(tick=120.0, channel=0, data=[42, 64]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[42, 100]),
   mydy.NoteOnEvent(tick=240.0, channel=0, data=[38, 100]),
   mydy.NoteOffEvent(tick=0.0, channel=0, data=[42, 64]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[42, 100]),
   mydy.NoteOffEvent(tick=120.0, channel=0, data=[38, 64]),
   mydy.NoteOffEvent(tick=120.0, channel=0, data=[42, 64]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=[42, 100]),
   mydy.NoteOffEvent(tick=30.0, channel=0, data=[42, 64]),
   mydy.NoteOnEvent(tick=0.0, channel=0, data=

## Reading notes

The following cell access the loaded MIDI file and reads the  ```Track```. Using the  ```mydy``` library, MIDI files are structured in the following way:

 ```Pattern -> Track -> MIDI_Events ```
 
 This means that, whenever I open a MIDI file, i will get a  ```Pattern ```, which contains  ```Track ```, which contains ```MIDI_Events ```.
 
 This scructure is used because a single MIDI file can contain multiple instruments (guitar, drums, bass ecc...), so each  ```Track ``` corresponds to an instrument, and the  ```MIDI_Events ``` are the note played by the single instrument.
 
 In our case we can assume to be working with single  ```Track ``` MIDI files (we're interested only in drums).
 
 **NB**: i'm using  ```track.make_ticks_abs()``` to convert the time (expressed in ticks/PPQ) from a relative value into an absolute one.<br>The standard representation of MIDI files represents note based on the time elapsed by the previous note. With this command i'm converting the time from relative to absolute (each note is represented with the time elapsed by the beginning of the song) 

In [33]:

track = test[0] #selecting the track (since it's only one, it will be always at index 0)
track_abs = track.make_ticks_abs()# Converting time from relative to an absolute measure


filtered_list=track_abs.filter(lambda e: isinstance(e, Events.NoteOnEvent))# Selects only Note_On events, i'm discarding the note off

#Just a bunch of test printings 
print(filtered_list)
print(len(filtered_list))
print(filtered_list[2].data[1])
print(test.resolution)


mydy.Track(relative: False\
  [mydy.NoteOnEvent(tick=1920.0, channel=0, data=[36, 100]),
   mydy.NoteOnEvent(tick=1920.0, channel=0, data=[42, 100]),
   mydy.NoteOnEvent(tick=2160.0, channel=0, data=[42, 100]),
   mydy.NoteOnEvent(tick=2400.0, channel=0, data=[38, 100]),
   mydy.NoteOnEvent(tick=2400.0, channel=0, data=[42, 100]),
   mydy.NoteOnEvent(tick=2640.0, channel=0, data=[42, 100]),
   mydy.NoteOnEvent(tick=2670.0, channel=0, data=[42, 100]),
   mydy.NoteOnEvent(tick=2700.0, channel=0, data=[42, 100]),
   mydy.NoteOnEvent(tick=2730.0, channel=0, data=[42, 100]),
   mydy.NoteOnEvent(tick=2760.0, channel=0, data=[36, 100]),
   mydy.NoteOnEvent(tick=2760.0, channel=0, data=[42, 100]),
   mydy.NoteOnEvent(tick=2880.0, channel=0, data=[42, 100]),
   mydy.NoteOnEvent(tick=3000.0, channel=0, data=[36, 100]),
   mydy.NoteOnEvent(tick=3120.0, channel=0, data=[42, 100]),
   mydy.NoteOnEvent(tick=3360.0, channel=0, data=[38, 100]),
   mydy.NoteOnEvent(tick=3360.0, channel=0, data=[42, 100

## Creating couples (Pitch, time)
So basically now i'm processing the previous raw data (as you can see from the printings, it's quite a mess) in order to obtain couples of ```(pitch, time)``` for each note.


In [34]:
notes = []
couple = ()

for element in filtered_list:
    
    ### test printings
    print(element.tick)
    print(element.data[0])
    ###
    
    notes.append(tuple((element.data[0],element.tick))) #storing couples of (note_pitch,tick_time)


print('printing tuples')
print(notes)


1920.0
36
1920.0
42
2160.0
42
2400.0
38
2400.0
42
2640.0
42
2670.0
42
2700.0
42
2730.0
42
2760.0
36
2760.0
42
2880.0
42
3000.0
36
3120.0
42
3360.0
38
3360.0
42
3600.0
42
3840.0
36
3840.0
42
3960.0
42
4080.0
42
4320.0
38
4320.0
42
4560.0
42
4680.0
36
4800.0
42
4920.0
36
5040.0
42
5280.0
38
5280.0
42
5520.0
38
5520.0
42
5550.0
42
5580.0
42
5610.0
42
5640.0
38
5640.0
42
5760.0
36
5760.0
42
6000.0
42
6240.0
38
6240.0
42
6480.0
42
6510.0
42
6540.0
42
6570.0
42
6600.0
36
6600.0
42
6720.0
42
6840.0
36
6960.0
42
7200.0
38
7200.0
42
7440.0
42
7680.0
36
7680.0
42
7800.0
42
7920.0
42
8160.0
38
8160.0
42
8400.0
42
8520.0
36
8520.0
42
8640.0
42
8760.0
36
8880.0
42
9120.0
38
9120.0
42
9360.0
38
9360.0
42
9390.0
42
9420.0
42
9450.0
42
9480.0
38
9480.0
42
9600.0
36
9600.0
42
9840.0
42
10080.0
38
10080.0
42
10320.0
42
10350.0
42
10380.0
42
10410.0
42
10440.0
36
10440.0
42
10560.0
42
10680.0
36
10800.0
42
11040.0
38
11040.0
42
11280.0
42
11520.0
36
11520.0
42
11640.0
42
11760.0
42
12000.0
38
12000.0
42


In [35]:
#RANDOM TEST ON THE LIBRARY, USELESS
test= Constants.C_3
print(test)

36


## Trouble and make it double
Now we're going into the tough part.

The following cell is an *utility* in order to convert our MIDI into a text file (that we will use to traing our network, read the reference model here: https://keunwoochoi.wordpress.com/2016/02/23/lstmetallica/)

Basically it creates 2 classes: ```Note``` and ```Note_List```.

**1)** *Note*:    Creates a simple ```Note``` object composed by ```pitch``` ```c_tick``` and ```idx```. ```pitch``` and ```c_tick``` are the already mentioned pitch and time, while ```idx``` is a variable that counts the index of my note on a 16-th note reference. <br>**For example**: if i have 2 whole notes, the second note will have ```idx=16```.

**2)** *Note_list*:  Creates an empty list where we will add our *Note* objects. It also contains supports methods used to create and manage this list.

**List of methods**: I'll add some comments to the code and make them more readable.


In [36]:
from mydy import Events, FileIO, Containers
import pdb

#Function responsible for converting midi notes into text. Since i have to train my network over the structure i decided
#which is 0b0000000 for no note, 0b01000000 for kick ecc... i need to convert midi notes into this format.

#The original script used for midi-text translation has been lost, must be re-implemented again
PPQ = 480 # Pulse per quater note. Used in sequencers. Standard value
event_per_bar = 32 # to quantise.
min_ppq = PPQ / (event_per_bar/4)

# ignore: 39 hand clap, 54 tambourine, 56 Cowbell, 58 Vibraslap, 60-81

#the dictionary below maps values to other ones. Reduced the size of the used notes. For example
#if i have an eletric snare or a stick snare, i just map both of them into a standard snare

drum_conversion = {35:36, # acoustic bass drum -> bass drum (36)
                    37:38, 40:38, # 37:side stick, 38: acou snare, 40: electric snare
                    43:41, # 41 low floor tom, 43 ghigh floor tom
                    47:45, # 45 low tom, 47 low-mid tom
                    50:48, # 50 high tom, 48 hi mid tom
                    44:42, # 42 closed HH, 44 pedal HH
                    57:49, # 57 Crash 2, 49 Crash 1
                    59:51, 53:51, 55:51, # 59 Ride 2, 51 Ride 1, 53 Ride bell, 55 Splash
                    52:49 # 52: China cymbal
                    }

#Used in the code to map elements, everything that has not one of the following number is discarded.
#Basically i'm ignoring notes that are not in my dataset (for examle i'll ignore shakers ecc...)
                # k, sn,cHH,oHH,LFtom,ltm,htm,Rde,Crash
allowed_pitch = [36, 38, 42, 46, 41, 45, 48, 51, 49] # 46: open HH
cymbals_pitch = [49, 51] # crash, ride
cymbals_pitch = [] # crash, ride
# pitch_to_midipitch = {36:midi.C_2, # kick # for general MIDI Drum map
# 						38:midi.D_2, # Snare
# 						39:midi.Eb_2, # hand clap (it's alive by mistake..)
# 						41:midi.F_2, # Low floor tom
# 						42:midi.Gb_2, # Close HH
# 						45:midi.A_2, # Low tom
# 						46:midi.Bb_2, # Open HH
# 						48:midi.C_3,  # Hi Mid Tom
# 						49:midi.Db_3, # Crash
# 						51:midi.Eb_3 # Ride
# 						}

#mapping midi values into notes
pitch_to_midipitch = {36:Constants.C_3,  # for logic 'SoCal' drum mapping
                        38:Constants.D_3, 
                        39:Constants.Eb_3,
                        41:Constants.F_3,
                        42:Constants.Gb_3,
                        45:Constants.A_3,
                        46:Constants.Bb_3,
                        48:Constants.C_4,
                        49:Constants.Db_4,
                        51:Constants.Eb_4
                        }
#la singola nota è un elemento composto da pitch (numerico, pitch midi) e tick (modo per tenere il tempo in midi)
class Note:
    def __init__(self, pitch, c_tick):
        self.pitch = pitch
        self.c_tick = c_tick # cumulated_tick of a midi note

    def add_index(self, idx):
        '''index --> 16-th note-based index starts from 0'''
        self.idx = idx

class Note_List():
    def __init__(self):
        ''''''
        self.notes = []
        self.quantised = False
        self.max_idx = None

    def add_note(self, note):
        '''note: instance of Note class'''
        self.notes.append(note)

    def quantise(self, minimum_ppq):
        '''
        e.g. if minimum_ppq=120, quantise by 16-th note.
        
        '''
        if not self.quantised:
            for note in self.notes:
                note.c_tick = ((note.c_tick+minimum_ppq/2)//minimum_ppq)* minimum_ppq # quantise
                #here the index is calculated. The index is an absolute index over the 16th notes.
                #for example an index of value 34, means that my current note appears after 34 chromes
                #it's simply calculated by dividing the cumulated tick of the note by the ticks contained in a 16th note
                note.add_index(note.c_tick/minimum_ppq)
            #NB: THE QUANTIZATION FUNCTION ITERATES OVER ALL THE NOTES. So first i add all the notes, then i iterate and quantize

            #Does this automatically reference to the last item added?
            #YES. The counter note will store the last element of the iteration. So basically here i'm assigning as max index the index of the last added note
            self.max_idx = note.idx

            #Here checks if if my ending is a full musical bar. For example, if my file ends with a single kick, i'll add that note.
            #but that kick will (probably) be at the beginning of the last musical bar. So i have to "pad" until the end.
            #It's like adding a pause on my piece, so i have all complete bars and no trucated ones at the end
            if (self.max_idx + 1) % event_per_bar != 0:
                self.max_idx += event_per_bar - ((self.max_idx + 1) % event_per_bar) # make sure it has a FULL bar at the end.
            self.quantised = True

        return

    def simplify_drums(self):
        ''' use only allowed pitch - and converted not allowed pitch to the similar in a sense of drums!
        '''
        #Here forces conversion into the pitches in drum_conversion
        for note in self.notes:
            if note.pitch in drum_conversion: # ignore those not included in the key
                note.pitch = drum_conversion[note.pitch]
        #https://stackoverflow.com/questions/30670310/what-do-brackets-in-a-for-loop-in-python-mean
        #The following one is a list comprehension. Basically generates a new list from an existing one using a given condition on the elements
        self.notes = [note for note in self.notes if note.pitch in allowed_pitch]	

        return

    def return_as_text(self):
        ''''''
        length = int(self.max_idx + 1) # of events in the track.
        #print(type(length))
        event_track = []
        #Thw following cycle create a 9 by N matrix. I append N times a vector of nine zeros.
        #This means that i create N notes, and then i initialize them with all zeros (9 zeros, since a note is represented by a 9 element binary number)

        for note_idx in range(length):  #sostituire xrange con range in Python3
            event_track.append(['0']*len(allowed_pitch))

        num_bars = length/event_per_bar# + ceil(len(event_texts_temp) % _event_per_bar)

        for note in self.notes:
            pitch_here = note.pitch
            #The following line returns the index of the passed pitch. Basically given an input generic pitch
            #it returns the associated pitch in my vocabolary (computes the actual mapping from the whole
            #vocabolary of notes into my reduced one)
            note_add_pitch_index = allowed_pitch.index(pitch_here) # 0-8
            #print(type(note.idx))  
            #print(type(note_add_pitch_index))
            event_track[int(note.idx)][note_add_pitch_index] = '1'
            # print note.idx, note.c_tick, note_add_pitch_index, ''.join(event_track[note.idx])
            # pdb.set_trace()

        event_text_temp = ['0b'+''.join(e) for e in event_track] # encoding to binary

        event_text = []
        # event_text.append('SONG_BEGIN')
        # event_text.append('BAR')
        print(num_bars)
        print(type(num_bars))        
        for bar_idx in range(int(num_bars)):
            event_from = bar_idx * event_per_bar
            event_to = event_from + event_per_bar
            event_text = event_text + event_text_temp[event_from:event_to]
            event_text.append('BAR')

        # event_text.append('SONG_END')

        return ' '.join(event_text)


## Creating Note_List
As simple as that, i'm taking my starting list of couples ```(pitch,time)``` into ```Note```  objects. 

Why? Because you should recycle also code, not only plastic. LUL



In [37]:

##NB: AGGIUNGERE GLI idx ALLE SINGLE NOTES! 
note_list = Note_List()
for note in notes:
    pitch = note[0]
    tick = note[1]
    idx = int(tick / min_ppq)
    new_note = Note(pitch,tick)
    new_note.add_index(idx)
    note_list.add_note(new_note)


In [38]:
for note in note_list.notes:
    print(note.idx)

32
32
36
40
40
44
44
45
45
46
46
48
50
52
56
56
60
64
64
66
68
72
72
76
78
80
82
84
88
88
92
92
92
93
93
94
94
96
96
100
104
104
108
108
109
109
110
110
112
114
116
120
120
124
128
128
130
132
136
136
140
142
142
144
146
148
152
152
156
156
156
157
157
158
158
160
160
164
168
168
172
172
173
173
174
174
176
178
180
184
184
188
192
192
194
196
200
200
204
206
208
210
212
216
216
220
220
220
221
221
222
222
224
224
228
232
232
236
236
237
237
238
238
240
242
244
248
248
252
256
256
258
260
264
264
268
270
270
272
274
276
280
280
284
284
284
285
285
286
286
288
288
292
296
296
300
300
301
301
302
302
304
306
308
312
312
316
320
320
322
324
328
328
332
334
336
338
340
344
344
348
348
348
349
349
350
350
352
352
356
360
360
364
364
365
365
366
366
368
370
372
376
376
380
384
384
386
388
392
392
396
398
398
400
402
404
408
408
412
412
412
413
413
414
414
416
416
420
424
424
428
428
429
429
430
430
432
434
436
440
440
444
448
448
450
452
456
456
460
462
464
466
468
472
472
476
476
476
477
477

In [39]:

print('printing ticks before quantization')
for note in note_list.notes:
    print(note.c_tick)
    
note_list.quantise(min_ppq)

print('printing ticks after quantization')
for note in note_list.notes:
    print(note.c_tick)

printing ticks before quantization
1920.0
1920.0
2160.0
2400.0
2400.0
2640.0
2670.0
2700.0
2730.0
2760.0
2760.0
2880.0
3000.0
3120.0
3360.0
3360.0
3600.0
3840.0
3840.0
3960.0
4080.0
4320.0
4320.0
4560.0
4680.0
4800.0
4920.0
5040.0
5280.0
5280.0
5520.0
5520.0
5550.0
5580.0
5610.0
5640.0
5640.0
5760.0
5760.0
6000.0
6240.0
6240.0
6480.0
6510.0
6540.0
6570.0
6600.0
6600.0
6720.0
6840.0
6960.0
7200.0
7200.0
7440.0
7680.0
7680.0
7800.0
7920.0
8160.0
8160.0
8400.0
8520.0
8520.0
8640.0
8760.0
8880.0
9120.0
9120.0
9360.0
9360.0
9390.0
9420.0
9450.0
9480.0
9480.0
9600.0
9600.0
9840.0
10080.0
10080.0
10320.0
10350.0
10380.0
10410.0
10440.0
10440.0
10560.0
10680.0
10800.0
11040.0
11040.0
11280.0
11520.0
11520.0
11640.0
11760.0
12000.0
12000.0
12240.0
12360.0
12480.0
12600.0
12720.0
12960.0
12960.0
13200.0
13200.0
13230.0
13260.0
13290.0
13320.0
13320.0
13440.0
13440.0
13680.0
13920.0
13920.0
14160.0
14190.0
14220.0
14250.0
14280.0
14280.0
14400.0
14520.0
14640.0
14880.0
14880.0
15120.0
15360.0
153

In [40]:
note_list.simplify_drums()

In [41]:
txt = note_list.return_as_text()

17.0
<class 'float'>


In [42]:
print(txt)

0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 0b000000000 BAR 0b101000000 0b000000000 0b000000000 0b000000000 0b001000000 0b000000000 0b000000000 0b000000000 0b011000000 0b000000000 0b000000000 0b000000000 0b001000000 0b001000000 0b101000000 0b000000000 0b001000000 0b000000000 0b100000000 0b000000000 0b001000000 0b000000000 0b000000000 0b000000000 0b011000000 0b000000000 0b000000000 0b000000000 0b001000000 0b000000000 0b000000000 0b000000000 BAR 0b101000000 0b000000000 0b001000000 0b000000000 0b001000000 0b000000000 0b000000000 0b000000000 0b011000000 0b000000000 0b000000000 0b000000000 0b001000000 0b000000000 0b100000000 0b000000000 0b001000000 0b000000000 0b100000

In [43]:
text_file = open("sample.txt", "w")
text_file.write(txt)
text_file.close()

## Testing Txt to MIDI conversion in the following cells

The following cell receives a single line from the txt file until the first 'BAR' element. ```encoded_drums``` is an array where each entry is a note in .txt format, so something like ```0xb011010100```.

```allowed_pitch``` has the following structure ```allowed_pitch = [36, 38, 42, 46, 41, 45, 48, 51, 49]``` where each entry is corresponds to a note in MIDI number (kick, snare ecc..., you can fin the exact notation inside the ```Note_list``` class).

So basically the following function iterates over the single txt note, and for each one of them, iterates over the single binary number and checks if it's equal to one. 
If so, it assigns the equivalent note value taken from ```allowed_pitch```. Basically is doing a 1on1 mapping between the single digit of the binary txt note and the single note taken from ```allowed_pitch```.

In [44]:
#Function that converts txt to notes. The note is represented as a number (in the MIDI scale)

#in encoded drums ho una riga intera dal file (quindi i vari 0xb00101110) 
def text_to_notes(encoded_drums, note_list=None):
    ''' 
    0b0000000000 0b10000000 ...  -> corresponding note. 
    '''
    if note_list == None:
        note_list = Note_List()
#https://www.programiz.com/python-programming/methods/built-in/enumerate enumerate mi ritorna coppie di (indice,valore) 
    for word_idx, word in enumerate(encoded_drums):
        c_tick_here = word_idx*min_ppq 

        for pitch_idx, pitch in enumerate(allowed_pitch):

            if word[pitch_idx+2] == '1':
                new_note = Note(pitch, c_tick_here)
                note_list.add_note(new_note)
    return note_list

The following function uses ```text_to_notes``` in order to fully convert .txt into MIDI. 

It receives as input the file name

In [45]:
import os

def conv_text_to_midi(filename):
    if os.path.exists(filename[:-4]+'.mid'):
        return
    f = open(filename, 'r')
    #These multiple readlines are actually useless. Need to check the output of the NN, but right now they're useless.
    #One single readline is enough
    #f.readline() # title
    #f.readline() # seed sentence
    #legge una riga intera dal file
    sentence = f.readline()
    #splitta gli elementi letti a ogni spazio.
    encoded_drums = sentence.split(' ')

    #find the first BAR

    first_bar_idx = encoded_drums.index('BAR')

    encoded_drums = encoded_drums[first_bar_idx:]
    try:
        encoded_drums = [ele for ele in encoded_drums if ele not in ['BAR', 'SONG_BEGIN', 'SONG_END', '']]
    except:
        pdb.set_trace()

    # prepare output
    note_list = Note_List()
    pattern = Containers.Pattern(fmt=0) #Don't know why there's an assertion in the code for fmt=0 if Pattern.len < 1
    track = Containers.Track()
    #??
    PPQ = 480
    min_ppq = PPQ / (event_per_bar/4)
    track.resolution = PPQ # ???? too slow. why??
    pattern.resolution = PPQ
    # track.resolution = 192
    pattern.append(track)

    velocity = 84
    duration = min_ppq*9/10  # it is easier to set new ticks if duration is shorter than _min_ppq

    note_list = text_to_notes(encoded_drums, note_list=note_list)

    max_c_tick = 0 
    not_yet_offed = [] # set of midi.pitch object
    print('entering for note_idx cycle')
    for note_idx, note in enumerate(note_list.notes[:-1]):
        # add onset
        tick_here = note.c_tick - max_c_tick
        pitch_here = pitch_to_midipitch[note.pitch]
        # if pitch_here in cymbals_pitch: # "Lazy-off" for cymbals 
        # 	off = midi.NoteOffEvent(tick=0, pitch=pitch_here)
        # 	track.append(off)

        on = Events.NoteOnEvent(tick=tick_here, velocity=velocity, pitch=pitch_here)
        track.append(on)
        max_c_tick = max(max_c_tick, note.c_tick)
        # add offset for something not cymbal

        # if note_list.notes[note_idx+1].c_tick == note.c_tick:
        # 	if pitch_here not in cymbals_pitch:
        # 	# 	not_yet_offed.append(pitch_here)

        # else:
        # check out some note that not off-ed.
        
        #in questo ciclo pare non ci entri mai. 
        for off_idx, waiting_pitch in enumerate(not_yet_offed):
            print(off_idx)
            if off_idx == 0:
                off = Events.NoteOffEvent(tick=duration, pitch=waiting_pitch)
                max_c_tick = max_c_tick + duration
            else:
                print('appending end note')
                off = Events.NoteOffEvent(tick=0, pitch=waiting_pitch)
            track.append(off)
            not_yet_offed = [] # set of midi.pitch object 

    # finalise
    if note_list.notes == []:
        print ('No notes in %s' % filename)
        return
        pdb.set_trace()
    note = note_list.notes[-1]
    tick_here = note.c_tick - max_c_tick
    pitch_here = pitch_to_midipitch[note.pitch]
    on = Events.NoteOnEvent(tick=tick_here, velocity=velocity, pitch=pitch_here)
    off = Events.NoteOffEvent(tick=duration, pitch=pitch_here)

    for off_idx, waiting_pitch in enumerate(not_yet_offed):
        off = Events.NoteOffEvent(tick=0, pitch=waiting_pitch)

    # end of track event
    eot = Events.EndOfTrackEvent(tick=1)
    track.append(eot)
    # print pattern
    #print(pattern)
    FileIO.write_midifile(filename[:-4]+'.mid', pattern)


In [46]:
conv_text_to_midi("sample.txt")


entering for note_idx cycle


In [31]:
pattern = Containers.Pattern(fmt=0)
print(pattern)

mydy.Pattern(format=0, resolution=220, tracks=\
[])
